# Data Summary
---
#### According to the Google Sheets file the Columns consist of: 

**1:** Age Range \
**2:** Industry \
**3:** Job Title\
**4:** Additional Job Context\
**5:** Annual Salary \
**6:** Additional Income (bonuses, overtime...)\
**7:** Currency (USD, GBP, EUR...)\
**8:** Other Column for additional currency inputs (one input INR)\
**9:** Context for Additional Income (bonus percentages)\
**10:** What Country You work in\
**11:** If U.S. what State they work in\
**12:** If U.S. what City they work in\
**13:** How Many Years of Professional Experience overall\
**14:** How Many Years of Professional Experience in this specific field\
**15:** Highest level of education completed\
**16:** Gender\
**17:** Race\
\
I will download the sheets file as a csv and import it as a dataframe (I renamed the columns for ease of use as they were labeled as the question asked in the original survey):

In [104]:
import pandas as pd

# read csv file
df = pd.read_csv('salarySurveyRenamed.csv')

# check the data
print(df.head())

# check data types
df.dtypes

            Timestamp ageRange                       industry  \
0  4/27/2021 11:02:10    25-34   Education (Higher Education)   
1  4/27/2021 11:02:22    25-34              Computing or Tech   
2  4/27/2021 11:02:38    25-34  Accounting, Banking & Finance   
3  4/27/2021 11:02:41    25-34                     Nonprofits   
4  4/27/2021 11:02:42    25-34  Accounting, Banking & Finance   

                                   jobTitle jobContext annualSalary  \
0        Research and Instruction Librarian        NaN       55,000   
1  Change & Internal Communications Manager        NaN       54,600   
2                      Marketing Specialist        NaN       34,000   
3                           Program Manager        NaN       62,000   
4                        Accounting Manager        NaN       60,000   

   addIncome currency otherCurrency addIncomeContext     workCountry  \
0        0.0      USD           NaN              NaN   United States   
1     4000.0      GBP           NaN   

Timestamp            object
ageRange             object
industry             object
jobTitle             object
jobContext           object
annualSalary         object
addIncome           float64
currency             object
otherCurrency        object
addIncomeContext     object
workCountry          object
usState              object
usCity               object
overallProExp        object
fieldExp             object
eduLevel             object
gender               object
race                 object
dtype: object

The dataframe consists of an additional timestamp column that will need to be removed during cleaning.

---

# Data Cleaning

### 1: Combining Salary and Additional Salary
- convert values to numeric int/float (addIncome is already a float, so just change salary)
- create a total compensation column (salary + additional)
- convert to one standard currency, USD

In [105]:
# check for nulls and fill if needed
print("NaNs in salary: ", df["annualSalary"].isna().sum())
print("NaNs in Additional Income: ",df["addIncome"].isna().sum())

# fill the NaNs in additional income with 0 so I can correctly calculate total compensation
df["addIncome"] = df["addIncome"].fillna(0)
print(df["addIncome"].head(5))

# check types
df.dtypes[["annualSalary", "addIncome"]]

# convert annualSalary to type float and remove commas
df["annualSalary"] = pd.to_numeric(df["annualSalary"].astype(str).str.replace(",", ""), errors="coerce")
print(df["annualSalary"].head(5))

# now calculate for total income
df["totalCompensation"] = df["annualSalary"] + df["addIncome"]
print(df["totalCompensation"].head(5))


# we have to standardize all values to USD
# combine currency and otherCurrency
df["currency"] = df["currency"].fillna(df["otherCurrency"])
print("NaNs in currency: ", df["currency"].isna().sum())

# drop otherCurrency
df = df.drop(columns=["otherCurrency"])

# find unique currencies to create dictionary for conversion
df["currency"] = df["currency"].str.strip().str.upper()
unique_currencies = df["currency"].dropna().unique()
print(unique_currencies)


NaNs in salary:  0
NaNs in Additional Income:  7332
0       0.0
1    4000.0
2       0.0
3    3000.0
4    7000.0
Name: addIncome, dtype: float64
0    55000
1    54600
2    34000
3    62000
4    60000
Name: annualSalary, dtype: int64
0    55000.0
1    58600.0
2    34000.0
3    65000.0
4    67000.0
Name: totalCompensation, dtype: float64
NaNs in currency:  0
['USD' 'GBP' 'CAD' 'EUR' 'AUD/NZD' 'OTHER' 'CHF' 'ZAR' 'SEK' 'HKD' 'JPY']


In [106]:
# dictionary for conversions, based on google 4/22/25:
conversion_rates = {
    "USD": 1.0,
    "GBP": 1.34,
    "CAD": 0.72,
    "EUR": 1.15,
    "AUD/NZD": 0.70,
    "CHF": 1.23,
    "ZAR": 0.054,
    "SEK": 0.11,
    "HKD": 0.13,
    "JPY": 0.0071,
}

# drop rows that answered other, as too many values and non standard answers
df = df[df["currency"].str.strip().str.upper() != "OTHER"]

# find unique currencies to create dictionary for conversion
df["currency"] = df["currency"].str.strip().str.upper()
print("NaNs in currency: ", df["currency"].isna().sum())
unique_currencies = df["currency"].dropna().unique()
print(unique_currencies)

# calculate USD based total income
df["compUSD"] = df.apply(
    lambda row: row["totalCompensation"] * conversion_rates.get(row["currency"], None),
    axis=1
)
print(df["compUSD"].head(5))
print("NaNs in compUSD: ", df["compUSD"].isna().sum())

NaNs in currency:  0
['USD' 'GBP' 'CAD' 'EUR' 'AUD/NZD' 'CHF' 'ZAR' 'SEK' 'HKD' 'JPY']
0    55000.0
1    78524.0
2    34000.0
3    65000.0
4    67000.0
Name: compUSD, dtype: float64
NaNs in compUSD:  0


# 2. Drop Unused Columns

- dropping columns such as timestamp, jobContext, jobTitle and additional income context
- we plan on generalizing based on industry to make it easier to group individuals

In [107]:
print(df.columns.tolist())

# drop timestamp
df = df.drop(columns=["Timestamp"])

df = df.drop(columns=["jobContext"])
df = df.drop(columns=["jobTitle"])

df = df.drop(columns=["addIncomeContext"])

df.head

['Timestamp', 'ageRange', 'industry', 'jobTitle', 'jobContext', 'annualSalary', 'addIncome', 'currency', 'addIncomeContext', 'workCountry', 'usState', 'usCity', 'overallProExp', 'fieldExp', 'eduLevel', 'gender', 'race', 'totalCompensation', 'compUSD']


<bound method NDFrame.head of       ageRange                       industry  annualSalary  addIncome  \
0        25-34   Education (Higher Education)         55000        0.0   
1        25-34              Computing or Tech         54600     4000.0   
2        25-34  Accounting, Banking & Finance         34000        0.0   
3        25-34                     Nonprofits         62000     3000.0   
4        25-34  Accounting, Banking & Finance         60000     7000.0   
...        ...                            ...           ...        ...   
28130    25-34   Engineering or Manufacturing         88000        0.0   
28131    25-34              Computing or Tech         46000        0.0   
28132    18-24                            NaN             0        0.0   
28133    25-34              Computing or Tech        100000    50000.0   
28134    25-34                            ABA         77000     3500.0   

      currency     workCountry         usState       usCity   overallProExp  \
0 

# Check NA's

- noticed NaNs in fields that would normally be filled, drop these data points as fields like gender cannot be artificially filled

In [111]:
df.isna().sum()

ageRange                0
industry                0
annualSalary            0
addIncome               0
currency                0
workCountry             0
usState              4757
usCity                 81
overallProExp           0
fieldExp                0
eduLevel                0
gender                  0
race                    0
totalCompensation       0
compUSD                 0
dtype: int64

In [109]:
# first drop all rows where industry is NA
df = df[df["industry"].notna()]

In [110]:
# drop all data points where eduLevel, gender and race are null
df = df[df["eduLevel"].notna()]
df = df[df["gender"].notna()]
df = df[df["race"].notna()]

# 3. Normalize Country Location

Make it so all values correlating to work in the United States are entered as USA and not united states, united states of america, us, usa...

I will do this for all countries as well


In [112]:
unique_countries = df["workCountry"].dropna().unique()
print(unique_countries)

['United States' 'United Kingdom' 'US' 'USA' 'Canada' 'United Kingdom '
 'usa' 'UK' 'Scotland ' 'U.S.' 'United States ' 'The Netherlands'
 'Australia ' 'Spain' 'us' 'Usa' 'England' 'finland'
 'United States of America' 'France' 'United states' 'Scotland' 'USA '
 'United states ' 'Germany' 'UK ' 'united states' 'Ireland' 'Australia'
 'Uk' 'United States of America ' 'U.S. ' 'canada' 'Canada ' 'U.S>' 'ISA'
 'Great Britain ' 'US ' 'United State' 'U.S.A' 'Denmark' 'U.S.A.'
 'America' 'Netherlands' 'netherlands' 'England '
 'united states of america' 'Ireland ' 'Switzerland' 'Netherlands '
 'Bermuda' 'Us' 'The United States' 'United State of America' 'Mexico '
 'United Stated' 'South Africa ' 'Belgium' 'Northern Ireland' 'u.s.'
 'South Africa' 'UNITED STATES' 'united States' 'Sweden' 'Hong Kong'
 'Kuwait' 'Sri lanka' 'Contracts' 'USA-- Virgin Islands' 'United Statws'
 'England/UK' 'U.S'
 "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work,

In [113]:
# Normalize US and drop all non US matches as data is heavily grouped towards United States

country_aliases = {
    # USA variants
    "UNITED STATES": "USA",
    "US": "USA",
    "USA": "USA",
    "U.S.": "USA",
    "U.S": "USA",
    "U.S.A": "USA",
    "U.S.A.": "USA",
    "UNITED STATES OF AMERICA": "USA",
    "UNITED STATES OF AMERICA ": "USA",
    "UNITED SATES OF AMERICA ": "USA",
    "UNITED SATES OF AMERICAN": "USA",
    "UNIITED STATES": "USA",
    "UNITED STATE": "USA",
    "UNITED STATED": "USA",
    "UNITED STATESP": "USA",
    "UNITED STATWS": "USA",
    "UNTED STATES": "USA",
    "UNITED STATTES": "USA",
    "UNITED STAES": "USA",
    "UNITED STATEA": "USA",
    "UNITED STATEES": "USA",
    "UNITED STARES": "USA",
    "UNITED STATUES": "USA",
    "USA-- VIRGIN ISLANDS": "USA",
    "AMERICA": "USA",
    "UNITED STATES IS AMERICA": "USA",
    "USAA": "USA",
    "USAB": "USA",
    "USS": "USA",
    "US OF A": "USA",
    "USA TOMORROW": "USA",
    "UNITED STATES (I WORK FROM HOME AND MY CLIENTS ARE ALL OVER THE US/CANADA/PR)": "USA",
    "UNITED STATES- PUERTO RICO": "USA",
    "USA, BUT FOR FOREIGN GOV'T": "USA",
    "U. S.": "USA",
    "U. S": "USA",
    "USA ": "USA",
    "US ": "USA",
    "Usa": "USA",
    "Us": "USA",
    "uSA": "USA",
    "usa": "USA",
    "uS": "USA",

    # UK variants
    "UNITED KINGDOM": "UK",
    "UK": "UK",
    "ENGLAND": "UK",
    "SCOTLAND": "UK",
    "WALES": "UK",
    "GREAT BRITAIN": "UK",
    "BRITAIN": "UK",
    "NORTHERN IRELAND": "UK",
    "UNITED KINGDOM (ENGLAND)": "UK",
    "ENGLAND, UK": "UK",
    "UK (NORTHERN IRELAND)": "UK",
    "UNITED KINGDOM.": "UK",
    "ENGLAND/UK": "UK",
    "SCOTLAND, UK": "UK",
    "U.K": "UK",
    "U.K.": "UK",
    "UK ": "UK",
    "UK, REMOTE": "UK",
    "ENGLAND, GB": "UK",
    "UK FOR U.S. COMPANY": "UK",
    "WALES (UK)": "UK",
    "WALES, UK": "UK",
    "UNITED KINGDOM ": "UK",
    "UNITED KINGDOMK": "UK",
    "UNITED KINDOM": "UK",
    "UNITED KINGDOM (WALES)": "UK",

    # Canada variants
    "CANADA": "CANADA",
    "CANADW": "CANADA",
    "CANADA ": "CANADA",
    "CANAD": "CANADA",
    "CANADA, OTTAWA, ONTARIO": "CANADA",
    "CAN": "CANADA",
    "CANA": "CANADA",
    "CSNADA": "CANADA",
    "CANDADA": "CANADA",
}


In [114]:
df["workCountry"] = df["workCountry"].str.strip().str.upper()

df["workCountry"] = df["workCountry"].replace(country_aliases)

print(df["workCountry"].value_counts().head(10))
print(df["workCountry"].unique())

workCountry
USA            22727
CANADA          1653
UK              1552
AUSTRALIA        377
GERMANY          190
NEW ZEALAND      119
IRELAND          117
FRANCE            65
NETHERLANDS       54
SPAIN             46
Name: count, dtype: int64
['USA' 'UK' 'CANADA' 'THE NETHERLANDS' 'AUSTRALIA' 'SPAIN' 'FINLAND'
 'FRANCE' 'GERMANY' 'IRELAND' 'U.S>' 'ISA' 'DENMARK' 'NETHERLANDS'
 'SWITZERLAND' 'BERMUDA' 'THE UNITED STATES' 'UNITED STATE OF AMERICA'
 'MEXICO' 'SOUTH AFRICA' 'BELGIUM' 'SWEDEN' 'HONG KONG' 'KUWAIT'
 'SRI LANKA' 'CONTRACTS'
 "WE DON'T GET RAISES, WE GET QUARTERLY BONUSES, BUT THEY PERIODICALLY ASSES INCOME IN THE AREA YOU WORK, SO I GOT A RAISE BECAUSE A 3RD PARTY ASSESSMENT SHOWED I WAS PAID TOO LITTLE FOR THE AREA WE WERE LOCATED"
 'UNITES STATES' 'ENGLAND, UK.' 'GREECE' 'UNITED SATES' 'AUSTRIA' 'GLOBAL'
 'UNITED STATES OF AMERICAN'
 'WORLDWIDE (BASED IN US BUT SHORT TERM TRIPS AROUDN THE WORLD)'
 'LUXEMBOURG' 'UNITED SATES OF AMERICA'
 'UNITED STATES (I WORK FROM HOME

In [115]:
# Based on the count of countries, we will focus on comparing USA, CANADA and UK
# I will now drop the unused countries from the data set
df = df[df["workCountry"].isin(["USA", "CANADA", "UK"])]
df.shape # adds correctly, 22726+1653+1552 = 25931

(25932, 15)

In [116]:
print(df["workCountry"].value_counts().head(10))
print(df["workCountry"].unique())

workCountry
USA       22727
CANADA     1653
UK         1552
Name: count, dtype: int64
['USA' 'UK' 'CANADA']


In [117]:
print(df.head)
df.dtypes
df.isna().sum()

<bound method NDFrame.head of       ageRange                       industry  annualSalary  addIncome  \
0        25-34   Education (Higher Education)         55000        0.0   
1        25-34              Computing or Tech         54600     4000.0   
2        25-34  Accounting, Banking & Finance         34000        0.0   
3        25-34                     Nonprofits         62000     3000.0   
4        25-34  Accounting, Banking & Finance         60000     7000.0   
...        ...                            ...           ...        ...   
28128    18-24                       research         31200        0.0   
28129    18-24   Engineering or Manufacturing         25000        0.0   
28130    25-34   Engineering or Manufacturing         88000        0.0   
28133    25-34              Computing or Tech        100000    50000.0   
28134    25-34                            ABA         77000     3500.0   

      currency workCountry         usState           usCity   overallProExp  \
0 

ageRange                0
industry                0
annualSalary            0
addIncome               0
currency                0
workCountry             0
usState              3358
usCity                 74
overallProExp           0
fieldExp                0
eduLevel                0
gender                  0
race                    0
totalCompensation       0
compUSD                 0
dtype: int64

In [118]:
# bin overallProExp and fieldExp
print(df["overallProExp"].head)
df["overallProExp"] = df["overallProExp"].str.replace(r"\s*-\s*", "-", regex=True)
print(df["overallProExp"].head)
print(df["overallProExp"].unique())

experience_bin_map = {
    "1 year or less": "0-1",
    "2-4 years": "2-4",
    "5-7 years": "5-7",
    "8-10 years": "8-10",
    "11-20 years": "11-20",
    "21-30 years": "21-30",
    "31-40 years": "31-40",
    "41 years or more": "41+"
}

<bound method NDFrame.head of 0             5-7 years
1          8 - 10 years
2           2 - 4 years
3          8 - 10 years
4          8 - 10 years
              ...      
28128    1 year or less
28129       2 - 4 years
28130         5-7 years
28133         5-7 years
28134         5-7 years
Name: overallProExp, Length: 25932, dtype: object>
<bound method NDFrame.head of 0             5-7 years
1            8-10 years
2             2-4 years
3            8-10 years
4            8-10 years
              ...      
28128    1 year or less
28129         2-4 years
28130         5-7 years
28133         5-7 years
28134         5-7 years
Name: overallProExp, Length: 25932, dtype: object>
['5-7 years' '8-10 years' '2-4 years' '21-30 years' '11-20 years'
 '1 year or less' '41 years or more' '31-40 years']


In [119]:
df["overallProExp"] = df["overallProExp"].map(experience_bin_map)
print(df["overallProExp"].head)
print(df["overallProExp"].unique())

<bound method NDFrame.head of 0         5-7
1        8-10
2         2-4
3        8-10
4        8-10
         ... 
28128     0-1
28129     2-4
28130     5-7
28133     5-7
28134     5-7
Name: overallProExp, Length: 25932, dtype: object>
['5-7' '8-10' '2-4' '21-30' '11-20' '0-1' '41+' '31-40']


In [120]:
df.isna().sum()

ageRange                0
industry                0
annualSalary            0
addIncome               0
currency                0
workCountry             0
usState              3358
usCity                 74
overallProExp           0
fieldExp                0
eduLevel                0
gender                  0
race                    0
totalCompensation       0
compUSD                 0
dtype: int64

In [121]:
# bin field exp
print(df["fieldExp"].head)
df["fieldExp"] = df["fieldExp"].str.replace(r"\s*-\s*", "-", regex=True)
print(df["fieldExp"].head)
print(df["fieldExp"].unique())

<bound method NDFrame.head of 0             5-7 years
1             5-7 years
2           2 - 4 years
3             5-7 years
4             5-7 years
              ...      
28128    1 year or less
28129       2 - 4 years
28130         5-7 years
28133       2 - 4 years
28134         5-7 years
Name: fieldExp, Length: 25932, dtype: object>
<bound method NDFrame.head of 0             5-7 years
1             5-7 years
2             2-4 years
3             5-7 years
4             5-7 years
              ...      
28128    1 year or less
28129         2-4 years
28130         5-7 years
28133         2-4 years
28134         5-7 years
Name: fieldExp, Length: 25932, dtype: object>
['5-7 years' '2-4 years' '21-30 years' '11-20 years' '1 year or less'
 '8-10 years' '31-40 years' '41 years or more']


In [122]:
df["fieldExp"] = df["fieldExp"].map(experience_bin_map)
print(df["fieldExp"].head)
print(df["fieldExp"].unique())

<bound method NDFrame.head of 0        5-7
1        5-7
2        2-4
3        5-7
4        5-7
        ... 
28128    0-1
28129    2-4
28130    5-7
28133    2-4
28134    5-7
Name: fieldExp, Length: 25932, dtype: object>
['5-7' '2-4' '21-30' '11-20' '0-1' '8-10' '31-40' '41+']


In [123]:
df.isna().sum()

ageRange                0
industry                0
annualSalary            0
addIncome               0
currency                0
workCountry             0
usState              3358
usCity                 74
overallProExp           0
fieldExp                0
eduLevel                0
gender                  0
race                    0
totalCompensation       0
compUSD                 0
dtype: int64

In [124]:
df.dtypes

ageRange              object
industry              object
annualSalary           int64
addIncome            float64
currency              object
workCountry           object
usState               object
usCity                object
overallProExp         object
fieldExp              object
eduLevel              object
gender                object
race                  object
totalCompensation    float64
compUSD              float64
dtype: object

---
# Summary

The dataframe has now been cleaned and is ready to be analyzed and worked on

What has been done consists of:
- dropping datapoints with missing values in crucial sections (industy, age, ...)
- grouped currency and other currency into one column just currency
- cleaning up total income
    - sum income and bonus
    - drop poorly formatted data (column for inputting other currency and some people wrote it in standard form but others wrote it out completely ex. INR vs indian rupee)
    - normalize total compensation to USD based on current exchange rates
    - stored in their own column (totalComepnsation and compUSD)
- isolated only USA, UK and CANADA and ensured consistency acrosss names (ie. United States -> USA...)
- cleaned up and binned experience level so that it is in numerical bins (1-2, 2-5,...) rather than having the word years, makes future processing much easier

\
The data should be similar to before just a little easier to read and manipulate for machinelearning models
- just know totalCompensation and compUSD are new columns
- totalCompensation is sum of annualSalary and addIncome
- compUSD is totalCompensation converted to USD based on currency column

In [125]:
# export the cleaned dataframe as csv
df.to_csv("cleandata.csv", index=False)

In [129]:
print(df.dtypes)



ageRange              object
industry              object
annualSalary           int64
addIncome            float64
currency              object
workCountry           object
usState               object
usCity                object
overallProExp         object
fieldExp              object
eduLevel              object
gender                object
race                  object
totalCompensation    float64
compUSD              float64
dtype: object


In [130]:
print(df.head)

<bound method NDFrame.head of       ageRange                       industry  annualSalary  addIncome  \
0        25-34   Education (Higher Education)         55000        0.0   
1        25-34              Computing or Tech         54600     4000.0   
2        25-34  Accounting, Banking & Finance         34000        0.0   
3        25-34                     Nonprofits         62000     3000.0   
4        25-34  Accounting, Banking & Finance         60000     7000.0   
...        ...                            ...           ...        ...   
28128    18-24                       research         31200        0.0   
28129    18-24   Engineering or Manufacturing         25000        0.0   
28130    25-34   Engineering or Manufacturing         88000        0.0   
28133    25-34              Computing or Tech        100000    50000.0   
28134    25-34                            ABA         77000     3500.0   

      currency workCountry         usState           usCity overallProExp  \
0   

In [131]:
df.isna().sum()

ageRange                0
industry                0
annualSalary            0
addIncome               0
currency                0
workCountry             0
usState              3358
usCity                 74
overallProExp           0
fieldExp                0
eduLevel                0
gender                  0
race                    0
totalCompensation       0
compUSD                 0
dtype: int64